In [1]:
import os
os.chdir('../')

In [22]:
model = YOLO(get_latest_model('artifacts/training'))

In [24]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    data_path: Path
    all_params: dict
    mlflow_uri: str
    model_name: str

In [14]:
from src.HowlerMonkey.constants import *
from src.HowlerMonkey.utils.common import read_yaml, create_directories, save_json, get_latest_model, clean_scores


In [15]:
print(os.environ["MLFLOW_TRACKING_USERNAME"])

GMainardi


In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            model_path = self.config.training.root_dir,
            data_path  = self.config.training.data_file_path,
            all_params = self.params,
            mlflow_uri = os.environ["MLFLOW_TRACKING_URI"],
            model_name = self.config.training.model_path.split('/')[-1].split('.')[0]
        )
        return evaluation_config
    

In [17]:
import pathlib as Path
import mlflow
import mlflow.pytorch
from ultralytics import YOLO
from urllib.parse import urlparse

In [27]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    @staticmethod
    def load_model(model_path: Path):
        return YOLO(get_latest_model(model_path))


    def evaluation(self):

        self.model = self.load_model(self.config.model_path)
        self.score = self.model.val(data=self.config.data_path).results_dict
        self.save_score()

    def save_score(self):
        self.scores = clean_scores(self.score)
        save_json(path="scores.json", data=self.scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                self.scores
            )
            # Model registry does not work with file store
            mlflow.log_artifact(get_latest_model(self.config.model_path), "model")


In [28]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-03-29 11:51:48,839: INFO: common] yaml file: config/config.yaml loaded successfully
[2024-03-29 11:51:48,840: INFO: common] yaml file: params.yaml loaded successfully
[2024-03-29 11:51:48,841: INFO: common] Creating directory: artifacts
Ultralytics YOLOv8.1.34 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /Users/guido/Library/CloudStorage/OneDrive-PUCRS-BR/Mestrado/Dissertacao/Howler-Monkey/artifacts/data_ingestion/data/labels/val.cache... 8 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


                   all          8         16      0.947        0.5      0.995      0.638
                  body          8          8      0.894          1      0.995      0.697
                  face          8          8          1          0      0.995      0.579
Speed: 1.1ms preprocess, 65.7ms inference, 0.0ms loss, 7.0ms postprocess per image
Results saved to runs/detect/val14
[2024-03-29 11:51:51,970: INFO: common] json file saved at: scores.json
